In [1]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 19.9 MB/s eta 0:00:00


In [2]:
import io
import ee
from google.colab import auth
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re
import geemap
import ipywidgets as widgets
import googleapiclient
from googleapiclient.discovery import build
from google.oauth2 import service_account
import geopandas as gpd
from osgeo import gdal, ogr, osr
import shutil
import rasterio
import joblib
import numpy as np
import pandas as pd
# Autentica e inizializza GEE
ee.Authenticate()
ee.Initialize(project='ee-andreagonnelli06')

In [5]:
# This cell and the next one are necessary just when we are using collab. otherwise, we need to put the address of the pre-trained model.
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/rf.pkl /content/

cp: cannot stat '/content/drive/MyDrive/rf.pkl': No such file or directory


In [23]:
import os
import ee
import geemap

ee.Initialize()
Map = geemap.Map()
Map
directory_path = '/content/Gdw'  # Replace with your desired directory path

# Create the directory if it does not exist
os.makedirs(directory_path, exist_ok=True)

coordinates = {
    "type": "Polygon",
    "coordinates": [
          [
            [
              10.205870738670399,
              44.75400228509341
            ],
            [
              10.205870738670399,
              44.73907269300071
            ],
            [
              10.226496908674022,
              44.73907269300071
            ],
            [
              10.226496908674022,
              44.75400228509341
            ],
            [
              10.205870738670399,
              44.75400228509341
            ]
          ]
        ]
}

rectangles = ee.Geometry.Polygon(coordinates["coordinates"])
Map.centerObject(rectangles)
# Funzione per ritagliare un'immagine in base al rettangolo di interesse


# Funzione per filtrare e unire le collezioni in base alle date
def filter_and_merge_collections(start_date, end_date):

    image = geemap.dynamic_world_s2(rectangles, start_date, end_date)\

    vis_params = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}
    #Map.addLayer(image, vis_params, "Sentinel-2 image")
    landcover = geemap.dynamic_world(rectangles, start_date, end_date, return_type="hillshade")\

    combined_image = landcover
    display(combined_image)

    return combined_image

# Funzione per creare le immagini medie mensili
def create_monthly_composites(start_year, end_year):
    monthly_composites = []

    for year in range(start_year, end_year + 1):
        for month in range(3, 6):
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')

            combined_image = filter_and_merge_collections(start_date, end_date)
            monthly_composites.append(combined_image.set('year', year).set('month', month))

    return ee.ImageCollection.fromImages(monthly_composites)

# Funzione per visualizzare le immagini medie mensili
def on_generate_button_clicked():
    start_year = 2024
    end_year = 2024

    global monthly_composites
    monthly_composites = create_monthly_composites(start_year, end_year)

    # Crea una mappa
    Map = geemap.Map()

    # Centra la mappa sull'area di interesse
    Map.centerObject(rectangles, zoom=8)

    # Aggiungi i layer delle immagini medie mensili alla mappa
    for year in range(start_year, end_year + 1):
        for month in range(3, 6):
            composite = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()
            layer_name = f'{year}-{month:02d} Composite'
    Map.addLayer(composite, {}, layer_name)
    Map.addLayer(rectangles, {}, 'Area of Interest')

    display(Map)

# Function to generate smaller bounding boxes within a larger region
def generate_tiles(region, tile_size):
    region = ee.Geometry(region)
    coords = region.bounds().getInfo()['coordinates'][0]
    min_lon, min_lat = coords[0]
    max_lon, max_lat = coords[2]

    tiles = []
    lat = min_lat

    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            tile = ee.Geometry.Rectangle([lon, lat, min(lon + tile_size, max_lon), min(lat + tile_size, max_lat)])
            tiles.append(tile)
            lon += tile_size
        lat += tile_size

    return tiles

# Function to download monthly composite images
def on_download_button_clicked():
    start_year = 2024
    start_month = 4
    end_year = 2024
    end_month = 4
    out_dir = 'Gdw'  # Local directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    # Define the approximate size of each tile in degrees
    tile_size = 0.22  # This value can be adjusted

    for year in range(start_year, end_year + 1):
        for month in range(start_month, end_month + 1):
            composite = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()

            file_name_prefix = f'{year}_{month:02d}'
            tiles = generate_tiles(rectangles, tile_size)  # Generate tiles

            for i, tile in enumerate(tiles):
                file_name = f'{file_name_prefix}_tile_{i:02d}.tif'
                file_path = os.path.join(out_dir, file_name)
                try:
                    geemap.ee_export_image(composite, filename=file_path, scale=10, region=tile)
                    print(f'{file_name} downloaded successfully.')
                except Exception as e:
                    print(f'Error downloading {file_name}: {e}')

# Esegui la funzione per generare e visualizzare le immagini
on_generate_button_clicked()

# Esegui la funzione per scaricare le immagini
on_download_button_clicked()


Map(center=[44.74653763176004, 10.216183823664526], controls=(WidgetControl(options=['position', 'transparent_…

Generating URL ...
Please wait ...
Data downloaded to /content/Gdw/2024_04_tile_00.tif
2024_04_tile_00.tif downloaded successfully.


In [20]:
Map.addLayer(rectangles)
Map

Map(center=[44.746537721079434, 10.21618382367221], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
import os
import json
from osgeo import gdal, ogr, osr
import numpy as np

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1'
]

def read_band_as_array(raster_path, band_index):
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        print(f"Unable to open {raster_path}")
        return None

    if band_index < 1 or band_index > src_ds.RasterCount:
        print(f"Band index {band_index} is out of range for {raster_path}")
        return None

    band = src_ds.GetRasterBand(band_index)
    arr = band.ReadAsArray()

    src_ds = None  # Close the dataset

    return arr

def create_pixel_polygons(image_path, field_names, colors):
    src_ds = gdal.Open(image_path)
    if src_ds is None:
        print(f"Unable to open {image_path}")
        return []

    transform = src_ds.GetGeoTransform()
    rows, cols = src_ds.RasterYSize, src_ds.RasterXSize
    num_bands = src_ds.RasterCount

    arrays = []
    valid_field_names = []
    valid_colors = []

    for band_index in range(1, num_bands + 1):
        arr = read_band_as_array(image_path, band_index)
        if arr is not None:
            arrays.append(arr)
            valid_field_names.append(field_names[band_index - 1])
            valid_colors.append(colors[band_index - 1])

    if not arrays:
        print("No valid bands found in the image")
        return []

    arrays = np.stack(arrays, axis=-1)
    max_indices = np.argmax(arrays, axis=-1)
    max_values = np.max(arrays, axis=-1)

    srs = osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    srs.SetWellKnownGeogCS("EPSG:4326")

    features = []
    for i in range(rows):
        for j in range(cols):
            class_index = max_indices[i, j]
            value = arrays[i, j, class_index]


            field_name = valid_field_names[class_index]
            color = valid_colors[class_index]

            minx = transform[0] + j * transform[1]
            maxx = transform[0] + (j + 1) * transform[1]
            miny = transform[3] + i * transform[5]
            maxy = transform[3] + (i + 1) * transform[5]

            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(minx, miny)
            ring.AddPoint(maxx, miny)
            ring.AddPoint(maxx, maxy)
            ring.AddPoint(minx, maxy)
            ring.AddPoint(minx, miny)

            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            poly.TransformTo(srs)

            feature = {
                "type": "Feature",
                "geometry": json.loads(poly.ExportToJson()),
                "properties": {
                    "class": field_name,
                    "value": float(value),
                    'color': f'#{color}'
                }
            }
            features.append(feature)

    return features

def process_images(image_path, output_geojson_path):
    field_names = [
        'water', 'trees', 'grass', 'flooded_vegetation',
        'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
    ]

    features = create_pixel_polygons(image_path, field_names, VIS_PALETTE)

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    # Write the combined GeoJSON
    with open(output_geojson_path, 'w') as f:
        f.write(json.dumps(geojson, indent=2))

    print(f"Combined GeoJSON created: {output_geojson_path}")

# Example usage
image_path = 'Gdw/2024_04_tile_00.tif'
output_geojson_path = 'Gdw/global_dym2.geojson'
process_images(image_path, output_geojson_path)


Combined GeoJSON created: Gdw/global_dym2.geojson
